In [50]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

import re
import javalang

from itertools import chain
from collections import Counter

In [247]:
def compute_confusion_matrix(
    y_true, # a vector of token_ids
    y_pred # a vector of token_ids
):
    target_counts = Counter(y_true)
    predicted_counts = Counter(y_pred)
    
    # hits: count all tokens both inside 'predicted' and 'target'
    true_positives = sum((target_counts & predicted_counts).values())

    # false alarms: count all tokens inside 'predicted', but missing in 'target'
    false_positives = sum((predicted_counts - target_counts).values())

    # misses: count all tokens inside 'target', but missing in 'predicted'
    false_negatives = sum((target_counts - predicted_counts).values())


    return {
        'true_positives': true_positives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
    }

cm = compute_confusion_matrix(
    y_true = ['transform', 'Search', 'Response'],
    y_pred = ['modify', 'Search', 'Response', 'Data'],
)

precision = cm['true_positives'] / (cm['true_positives'] + cm['false_positives'])
recall = cm['true_positives'] / (cm['true_positives'] + cm['false_negatives'])
f1 = 2 * precision * recall / (precision + recall)

print(cm)
print({ 'precision': precision, 'recall': recall, 'f1': f1 })

{'true_positives': 2, 'false_positives': 2, 'false_negatives': 1}
{'precision': 0.5, 'recall': 0.6666666666666666, 'f1': 0.5714285714285715}


In [193]:
def compute_metrics(
    y_true, # a vector of token_ids
    y_pred # a vector of token_ids
):
    overlapping = Counter(y_true) & Counter(y_pred)
    overlapping_count = sum(overlapping.values())
    
    precision = 1.0 * overlapping_count / len(y_pred)
    recall = 1.0 * overlapping_count / len(y_true)
    f1 = (2.0 * precision * recall) / (precision + recall)

    return { 'precision': precision, 'recall': recall, 'f1': f1 }

compute_metrics(
    y_true = ['transform', 'Search', 'Response'],
    y_pred = ['modify', 'Search', 'Response', 'Data'],
)

{'precision': 0.5, 'recall': 0.6666666666666666, 'f1': 0.5714285714285715}

In [200]:
compute_metrics(
    y_true = [1, 2, 3, 4],
    y_pred = [4, 3, 2, 1],
)

{'precision': 1.0, 'recall': 1.0, 'f1': 1.0}

In [240]:
compute_metrics(
    y_true = [1, 0, 0],
    y_pred = [4, 3, 2, 1],
)

{'precision': 0.25, 'recall': 0.3333333333333333, 'f1': 0.28571428571428575}

In [235]:
a = tf.constant([34, 56, 12])
b = tf.constant([56])
intersection = tf.sets.intersection(a[None,:],b[None,:])
# tf.sparse_tensor_to_dense(intersection)

intersection.values.numpy()

array([56], dtype=int32)

In [237]:
tf.sets.intersection(y_true[None,0],y_pred[None,0]).values

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5], dtype=int32)>

In [2]:
df = pd.read_hdf('../data/interim/preprocessed/sequences.h5', key='data')

In [229]:
y_true = tf.constant(
    [[1, 2, 3, 4, 5, 0, 0, 0],
     [4, 3, 2, 1, 0, 0, 0, 0]],
    dtype=tf.int32
)

y_pred = tf.constant(
    [[1, 2, 5, 3, 4, 0, 0, 0],
     [4, 2, 2, 1, 0, 0, 0, 0]],
    dtype=tf.int32
)

(y_true, y_pred)

(<tf.Tensor: shape=(2, 8), dtype=int32, numpy=
 array([[1, 2, 3, 4, 5, 0, 0, 0],
        [4, 3, 2, 1, 0, 0, 0, 0]], dtype=int32)>,
 <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
 array([[1, 2, 5, 3, 4, 0, 0, 0],
        [4, 2, 2, 1, 0, 0, 0, 0]], dtype=int32)>)

In [230]:
# remove padding
y_true_ragged = tf.RaggedTensor.from_tensor(y_true, padding=0)
y_true_ragged

<tf.RaggedTensor [[1, 2, 3, 4, 5], [4, 3, 2, 1]]>

In [231]:
# pick from y_pred only these items which the y_true decides to pick
y_pred_ragged = tf.ragged.boolean_mask(y_pred, tf.cast(y_true, dtype=tf.bool))
y_pred_ragged

<tf.RaggedTensor [[1, 2, 5, 3, 4], [4, 2, 2, 1]]>

In [232]:
recall = tf.keras.metrics.Recall()
recall.update_state(
    y_true=tf.one_hot(y_true_ragged.to_tensor(), depth=10, axis=-1)[0],
    y_pred=tf.one_hot(y_pred_ragged.to_tensor(), depth=10, axis=-1)[0],
)
recall.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.4>

In [233]:
precision = tf.keras.metrics.Precision()

precision.update_state(
    y_true=tf.one_hot(y_true_ragged.to_tensor(), depth=10, axis=-1)[0],
    y_pred=tf.one_hot(y_pred_ragged.to_tensor(), depth=10, axis=-1)[0],
)

precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.4>

In [234]:
accuracy = tf.keras.metrics.Accuracy()

accuracy.update_state(
    y_true=tf.one_hot(y_true_ragged.to_tensor(), depth=10, axis=-1)[0],
    y_pred=tf.one_hot(y_pred_ragged.to_tensor(), depth=10, axis=-1)[0],
)

accuracy.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.88>

In [35]:
tf.one_hot(y_true, depth=10, axis=-1)

<tf.Tensor: shape=(2, 7, 10), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)>

In [49]:
y_true * (y_true != 0)

InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a int64 tensor but is a bool tensor [Op:Mul] name: mul/

In [6]:
preds = [
    [0.9, 0.1, 0],
    [0.2, 0.6, 0.2],
    [0, 0, 1],
    [0.4, 0.3, 0.3],
    [0, 0.9, 0.1],
    [0, 0, 1],
]
actuals = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1]]

In [4]:
precision = tf.metrics.Precision()

In [30]:
# is there a way to replace tf.one_hot with sparse_tensor? why hasn't anyone though of producing a SparseTensor when using tf.one_hot
# however, even if you pass a sparse tensor to update_state, it isn't smart enough yet to accept it
# TODO: can we ignore the matching zeros? they are most probably ignore in research papers
precision.update_state(
    y_pred=tf.one_hot(y_pred, depth=10, axis=-1), # preds
    y_true=tf.one_hot(y_true, depth=10, axis=-1)# actuals
)

In [29]:
precision.result().numpy()

0.78571427

In [25]:
precision.reset_states()

In [27]:
tf.sets.intersection(y_true, y_pred).numpy()

AttributeError: 'SparseTensor' object has no attribute 'numpy'

In [9]:
tf.one_hot(y_true, 256, axis = -1)

<tf.Tensor: shape=(2, 7, 256), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>

In [5]:
metric = tfa.metrics.F1Score(
    num_classes=6000,
    average=None, # TODO: can be also 'micro', 'macro' or 'weighted'
    # Elements of y_pred above threshold are considered to be 1, and the rest 0.
    # If threshold is None, the argmax is converted to 1, and the rest 0.
    threshold=None,
)

In [6]:
metric.update_state(
    y_true=y_true,
    y_pred=y_pred,
)

ValueError: in converted code:

    /home/tony/source/identifier-suggestion/.venv/lib/python3.7/site-packages/tensorflow_addons/metrics/f_scores.py:141 update_state  *
        self.true_positives.assign_add(_count_non_zero(y_pred * y_true))
    /home/tony/source/identifier-suggestion/.venv/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:786 assign_add
        name=name)
    /home/tony/source/identifier-suggestion/.venv/lib/python3.7/site-packages/tensorflow_core/python/ops/gen_resource_variable_ops.py:56 assign_add_variable_op
        "AssignAddVariableOp", resource=resource, value=value, name=name)
    /home/tony/source/identifier-suggestion/.venv/lib/python3.7/site-packages/tensorflow_core/python/framework/op_def_library.py:742 _apply_op_helper
        attrs=attr_protos, op_def=op_def)
    /home/tony/source/identifier-suggestion/.venv/lib/python3.7/site-packages/tensorflow_core/python/framework/func_graph.py:595 _create_op_internal
        compute_devi

In [136]:
np.stack(df['inputs'].head(2))

array([[   8,   15,  486,   10,    7,    5,  137, 4043,    4,  140,    3,
        1674,  259,   48,   25,    4,   58,    2,    5,  137, 4043,    4,
         140,  132,    3,    7,   17,  486,    2,    5, 1144,  609, 2062,
           3,  486,    2,    5,    9,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [99]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [35]:
modifiers = ['public', 'private', 'protected', 'static']

RE_WORDS = re.compile(r'''
    # Find words in a string. Order matters!
    [A-Z]+(?=[A-Z][a-z]) |  # All upper case before a capitalized word
    [A-Z]?[a-z]+ |  # Capitalized words / all lower case
    [A-Z]+ |  # All upper case
    \d+ | # Numbers
    .+
''', re.VERBOSE)

def split_subtokens(str):
    return [subtok for subtok in RE_WORDS.findall(str) if not subtok == '_']

In [36]:
def tokenize_method(method_body):
    method_content = method_body
    try:
        tokens = list(javalang.tokenizer.tokenize(method_content))
    except:
        print('ERROR in tokenizing: ' + method_content)
        #tokens = method_content.split(' ')
    if len(tokens) > 0:
        return ' '.join([' '.join(split_subtokens(i.value)) for i in tokens if not i.value in modifiers])
    else:
        return ''

In [38]:
tokenize_method('void (String fooBar){System.out.println("hello world");}')

'void ( String foo Bar ) { System . out . println ( "hello world" ) ; }'

In [106]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=1000,
    filters='',
    lower=False,
    oov_token='<OOV>',
)

In [107]:
sequences = [
    ['{', 'printf', '(', '"', 'hello', 'world', '"', ')', ';', '}'],
    ['{', 'fprintf', '(', '"', 'hello', 'dad', '"', ')', ';', '}'],
    ['{', 'vprintf', '(', '"', 'hello', 'mom', '"', ')', ';', '}'],
]

texts = [
    'void ( String foo Bar ) { System . out . println ( " hello world " ) ; }'
]

In [108]:
tokenizer.fit_on_texts(chain.from_iterable(sequences))

In [90]:
tokenizer.num_words = 5

In [95]:
tokenizer.word_index

{'<OOV>': 1,
 '"': 2,
 '{': 3,
 '(': 4,
 'hello': 5,
 ')': 6,
 ';': 7,
 '}': 8,
 'printf': 9,
 'world': 10,
 'fprintf': 11,
 'dad': 12,
 'vprintf': 13,
 'mom': 14}

In [104]:
list(map(lambda l: flatten(l), sequences))
sequences[0]

['{', 'printf', '(', '"', 'hello', 'world', '"', ')', ';', '}']

In [105]:
tokenizer.texts_to_sequences(map(lambda l: ' '.join(l), sequences))

[[3, 9, 4, 2, 5, 10, 2, 6, 7, 8],
 [3, 11, 4, 2, 5, 12, 2, 6, 7, 8],
 [3, 13, 4, 2, 5, 14, 2, 6, 7, 8]]

In [117]:
df = pd.read_hdf('../data/interim/preprocessed/sequences.h5', key='data')

In [128]:
dataset = tf.data.Dataset.from_tensor_slices(np.stack(df['inputs'].values))
dataset

<TensorSliceDataset shapes: (200,), types: tf.int64>

In [149]:
shifted = next(iter(dataset.map(lambda seq: seq[1:])))
tf.concat(axis=0, values=[shifted, [53100]])

<tf.Tensor: shape=(200,), dtype=int64, numpy=
array([61674, 16275, 29012, 40249, 25223,  6673, 15050, 59831, 11591,
       14256, 15050, 39792, 68638, 25223,  6673, 15050, 32537, 11591,
       40249, 33282, 16275, 68638, 25223, 60298, 11591, 16275, 68638,
       25223, 32001, 53100, 53100, 53100, 53100, 53100, 53100, 53100,
       53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100,
       53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100,
       53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100,
       53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100,
       53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100,
       53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100,
       53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100,
       53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100,
       53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100, 53100,
       53100, 53100, 53100, 53100, 53100, 53

In [146]:

tf.concat(values=[tf.constant(np.array([512])), [256]], axis=0)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([512, 256])>

In [129]:
[1, 2, 3, 4, 5][:-1]

[1, 2, 3, 4]

In [78]:
sentences = [
    'We pad the sequece in the post-order where maxlen is max_seq_length.',
    'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.',
]

<TensorSliceDataset shapes: (), types: tf.string>

b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.'
b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.'
b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.'
b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.'
b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.'
b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we tr

22

In [114]:
# 1. create Dataset entity
dataset = tf.data.Dataset.from_tensor_slices(sentences * 10)

print(dataset)
print()
for item in dataset:
    print(item.numpy())

# 2. Tokenize
# 3. Build vocabulary

# TODO: swap out the tokenizer with the javalang tokenizer
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor in dataset.take(2):
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

# 4. Encode the text/tokens into numbers

# TODO: this is very bad design by TF, because it includes both tokenization AND encoding
# we already did the tokenization in the last step ...
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

example_text = next(iter(dataset)).numpy()
print(example_text)
encoded_example = encoder.encode(example_text)
print(encoded_example)

for text_tensor in dataset.take(10):
    print(encoder.encode(text_tensor.numpy()))

def encode(text_tensor):
    print('--->', text_tensor.numpy())
    encoded_text = encoder.encode(text_tensor.numpy())
    print('<---', encoded_text)
    
    # TODO: why the fuck do we need to return a fucking list/tuple here?
    return [encoded_text]

def encode_map_fn(text):
    # py_func doesn't set the shape of the returned tensors.
    # TODO: but why?!?
    encoded_text = tf.py_function(encode, inp=[text], Tout=tf.int64, name='encode')
    
    print(f'<- encoded_text after py_function: {encoded_text}')
    
    # `tf.data.Datasets` work best if all components have a shape set
    #  so set the shapes manually: 
    # TODO: but why?!?
#     encoded_text.set_shape([None])

    return encoded_text

dataset = dataset.map(encode_map_fn)

print(dataset)
print()
for item in dataset.take(5):
    print(item.numpy())

# # 5. pad sequences
# def pad_seq(seq):
#     """
#         We pad the sequece in the post-order where maxlen is max_seq_length.
#         If any vector is larger than max_seq_length, we truncate the post-sequence
#     """
#     return tf.keras.preprocessing.sequence.pad_sequences(
#         [seq], # TODO: [seq.numpy()]?
#         maxlen=max_seq_length,
#         truncating='post',
#         padding='post',
#         value='',
# #         dtype=np.float
#     ).squeeze() # TODO: why sequeeze?

# # def pad_map_fn(seq):
# #     return tf.py_function(pad_seq, inp=[seq], Tout=(tf.float32))
# def pad_seq_map_fn(seq):
#     seq.numpy()

# dataset = dataset.map(pad_seq_map_fn)
    
# 6. one-hot encode
dataset = dataset.map(lambda seq: tf.one_hot(seq, vocab_size))


print(dataset)
print()
for item in dataset.take(5):
    print(item.numpy())

<TensorSliceDataset shapes: (), types: tf.string>

b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.'
b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.'
b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.'
b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.'
b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we truncate the post-sequence to shorten it.'
b'We pad the sequece in the post-order where maxlen is max_seq_length.'
b'If any vector is larger than max_seq_length, we tr

In [12]:
label1 = ['transform', 'Search', 'Response']
label2 = ['get', 'Abstract', 'Factory', 'Creator', 'Service']
labels = [label1, label2]

In [24]:
max_seq_length = 4

In [28]:
def pad_seq(seq):
    """
        We pad the sequece in the post-order where maxlen is max_seq_length.
        If any vector is larger than max_seq_length, we truncate the post-sequence
    """
    return tf.keras.preprocessing.sequence.pad_sequences(
        [seq], # TODO: [seq.numpy()]?
        maxlen=max_seq_length,
        truncating='post',
        padding='post',
        value='',
#         dtype=np.float
    ).squeeze() # TODO: why sequeeze?

# def pad_map_fn(seq):
#     return tf.py_function(pad_seq, inp=[seq], Tout=(tf.float32))

In [20]:
labels_tensor = tf.ragged.constant(labels)
labels_tensor

<tf.RaggedTensor [[b'transform', b'Search', b'Response'], [b'get', b'Abstract', b'Factory', b'Creator', b'Service']]>

In [22]:
label_vocabulary = {token for label in labels for token in label}
label_vocabulary

label_to_index = { index: token for token, index in enumerate(label_vocabulary) }
label_to_index

{'Service': 0,
 'get': 1,
 'Response': 2,
 'Creator': 3,
 'transform': 4,
 'Abstract': 5,
 'Factory': 6,
 'Search': 7}

In [32]:
labels_tensor.to_tensor()

<tf.Tensor: shape=(2, 5), dtype=string, numpy=
array([[b'transform', b'Search', b'Response', b'', b''],
       [b'get', b'Abstract', b'Factory', b'Creator', b'Service']],
      dtype=object)>

In [33]:
# 2. From RaggedTensors to normal Tensors
pad_seq(labels_tensor.to_tensor().numpy())

ValueError: `dtype` int32 is not compatible with `value`'s type: <class 'str'>
You should set `dtype=object` for variable length strings.

In [37]:

# first you have to convert the RaggedTensor to a normal tensor
# ValueError: TypeError: object of type 'RaggedTensor' has no len()
# tf.map_fn(lambda token: label_to_index[token], labels_tensor.to_tensor())
tf.py_function(lambda token: label_to_index[token], [labels_tensor.to_tensor()], tf.int64)

InvalidArgumentError: TypeError: Tensor is unhashable if Tensor equality is enabled. Instead, use tensor.experimental_ref() as the key.
Traceback (most recent call last):

  File "/home/tony/source/identifier-suggestion/.venv/lib/python3.7/site-packages/tensorflow_core/python/ops/script_ops.py", line 234, in __call__
    return func(device, token, args)

  File "/home/tony/source/identifier-suggestion/.venv/lib/python3.7/site-packages/tensorflow_core/python/ops/script_ops.py", line 123, in __call__
    ret = self._func(*args)

  File "<ipython-input-37-7853e3c76557>", line 4, in <lambda>
    tf.py_function(lambda token: label_to_index[token], [labels_tensor.to_tensor()], tf.int64)

  File "/home/tony/source/identifier-suggestion/.venv/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py", line 705, in __hash__
    raise TypeError("Tensor is unhashable if Tensor equality is enabled. "

TypeError: Tensor is unhashable if Tensor equality is enabled. Instead, use tensor.expe

In [18]:
tf.one_hot(labels_tensor, len(label_vocabulary))

NotFoundError: Could not find valid device for node.
Node:{{node OneHot}}
All kernels registered for op OneHot :
  device='XLA_CPU'; TI in [DT_INT32, DT_UINT8, DT_INT64]; T in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, ..., DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='XLA_CPU_JIT'; TI in [DT_INT32, DT_UINT8, DT_INT64]; T in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, ..., DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='XLA_GPU_JIT'; TI in [DT_INT32, DT_UINT8, DT_INT64]; T in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, ..., DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='CPU'; TI in [DT_INT64]; T in [DT_VARIANT]
  device='CPU'; TI in [DT_INT32]; T in [DT_VARIANT]
  device='CPU'; TI in [DT_UINT8]; T in [DT_VARIANT]
  device='CPU'; TI in [DT_INT64]; T in [DT_RESOURCE]
  device='CPU'; TI in [DT_INT32]; T in [DT_RESOURCE]
  device='CPU'; TI in [DT_UINT8]; T in [DT_RESOURCE]
  device='CPU'; TI in [DT_INT64];